# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import string
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# Load Datasets

In [2]:
#change the path of the files as per your system
aisles=pd.read_csv(r"C:\Users\Arpita\Downloads\instacart-market-basket-analysis\Instakart\aisles.csv")
dept=pd.read_csv(r"C:\Users\Arpita\Downloads\instacart-market-basket-analysis\Instakart\departments.csv")
order=pd.read_csv(r"C:\Users\Arpita\Downloads\instacart-market-basket-analysis\Instakart\orders.csv")
prod=pd.read_csv(r"C:\Users\Arpita\Downloads\instacart-market-basket-analysis\Instakart\products.csv")
sales_prior=pd.read_csv(r"C:\Users\Arpita\Downloads\instacart-market-basket-analysis\Instakart\order_products__prior.csv")
ord_train=pd.read_csv(r"C:\Users\Arpita\Downloads\instacart-market-basket-analysis\Instakart\order_products__train.csv")

# EDA

In [3]:
print(order.isnull().sum())

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64


In [4]:
daymed=(order['days_since_prior_order'].median())
print('Mean of days sice last order = ', daymed)
print('\n\n')
order['days_since_prior_order']=order['days_since_prior_order'].fillna(daymed)
print(order.isnull().sum())

Mean of days sice last order =  7.0



order_id                  0
user_id                   0
eval_set                  0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
dtype: int64


## Feature Engineering Users

In [5]:
# Creating a dataset with only the prior orders

# This ensures only the entries that are common to both the tables are included, and similar 
# order ids can be counted multiple times if they have different product ids

op = pd.merge(order, sales_prior, on='order_id', how='inner')
op.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,7.0,196,1,0
1,2539329,1,prior,1,2,8,7.0,14084,2,0
2,2539329,1,prior,1,2,8,7.0,12427,3,0
3,2539329,1,prior,1,2,8,7.0,26088,4,0
4,2539329,1,prior,1,2,8,7.0,26405,5,0


In [6]:
# We calculate the total number of orders each user has placed

users = op.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [7]:
# Now we get the average number of products each user buys including all their orders

#1. First we get the total number of products per user per order

total_prd_per_order = op.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()
total_prd_per_order.head(10)

,user_id,order_id,total_products_per_order
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6
5,1,2398795,6
6,1,2539329,5
7,1,2550362,9
8,1,3108588,6
9,1,3367565,4


In [8]:
#2. Now we get the average number of products purchased by each user overall

avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [9]:
# Deleting the total products column which is no longer needed

del total_prd_per_order

In [10]:
# Calculating which day of the week does each user place most orders

dow = op.groupby(by='user_id')['order_dow'].agg(lambda x: stats.mode(x)[0]).to_frame('dow_most_orders_u').reset_index()
dow.head()


#user1: 1,1,1,0,7,7,7,1,1 --> 1 , count : 9, max : 7
#user2: 2,2,6,6,6,6,6,6,0 --> 6

,user_id,dow_most_orders_u
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [11]:
# Calculating which hour of the day does each user place most orders

hod = op.groupby(by='user_id')['order_hour_of_day'].agg(lambda x: (stats.mode(x)[0])).to_frame('hod_most_orders_u').reset_index()
hod.head()


,user_id,hod_most_orders_u
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [12]:
# Now we merge all the features that we just extracted from our datset, one by one

#1. Merging avg products per users with users ids

users = users.merge(avg_products, on='user_id', how='left')
del avg_products
users.head()

,user_id,u_num_of_orders,u_avg_prd
0,1,10,5.900000
1,2,14,13.928571
2,3,12,7.333333
3,4,5,3.600000
4,5,4,9.250000


In [13]:
#2. Merging day of maximum purchase with database obtained in last step

users = users.merge(dow, on='user_id', how='left')
#deleting dow
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,10,5.900000,4
1,2,14,13.928571,2
2,3,12,7.333333,0
3,4,5,3.600000,4
4,5,4,9.250000,3


In [14]:
#3. Merging hour of maximum purchase with database produced in last step

users = users.merge(hod, on='user_id', how='left')
del hod
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u
0,1,10,5.900000,4,7
1,2,14,13.928571,2,9
2,3,12,7.333333,0,16
3,4,5,3.600000,4,15
4,5,4,9.250000,3,18


## Feature Engineering Products

In [15]:
# Number of times a particluar product was purchased

prd = op.groupby(by='product_id')['order_id'].agg('count').to_frame('prd_count_p').reset_index()
prd.head()

,product_id,prd_count_p
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [16]:
# Reorder ratio for each product

reorder_p = op.groupby(by='product_id')['reordered'].agg('mean').to_frame('p_reordered_ratio').reset_index()
reorder_p.head()

,product_id,p_reordered_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [17]:
# Merging the reorder ratio with product ids

prd = prd.merge(reorder_p, on='product_id', how='left')

#deleting reorder_p

del reorder_p
prd.head()

,product_id,prd_count_p,p_reordered_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.600000


## Creating user product features

In [18]:
# How many times has a user bought a particular product

uxp = op.groupby(by=['user_id', 'product_id'])['order_id'].agg('count').to_frame('uxp_times_bought').reset_index()
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [19]:
# Reorder ratio of each user for each product

reorder_uxp = op.groupby(by=['user_id', 'product_id'])['reordered'].agg('mean').to_frame('uxp_reordered_ratio').reset_index()
reorder_uxp.head()

,user_id,product_id,uxp_reordered_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [20]:
# Merging the user-product and product-reorder dataframes into one

uxp = uxp.merge(reorder_uxp, on=['user_id', 'product_id'], how='left')

del reorder_uxp
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio
0,1,196,10,0.900000
1,1,10258,9,0.888889
2,1,10326,1,0.000000
3,1,12427,10,0.900000
4,1,13032,3,0.666667


## Merging Final

In [21]:
# Merging users dataframe into the dataframe we obtianed in our last step

data = uxp.merge(users, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u
0,1,196,10,0.900000,10,5.9,4,7
1,1,10258,9,0.888889,10,5.9,4,7
2,1,10326,1,0.000000,10,5.9,4,7
3,1,12427,10,0.900000,10,5.9,4,7
4,1,13032,3,0.666667,10,5.9,4,7


In [22]:
# Merging products dataframe into the existing dataframe

data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio
0,1,196,10,0.900000,10,5.9,4,7,35791,0.776480
1,1,10258,9,0.888889,10,5.9,4,7,1946,0.713772
2,1,10326,1,0.000000,10,5.9,4,7,5526,0.652009
3,1,12427,10,0.900000,10,5.9,4,7,6476,0.740735
4,1,13032,3,0.666667,10,5.9,4,7,3751,0.657158


In [23]:
# Deleting columns that are no longer significant

del [users, prd, uxp]